<a href="https://colab.research.google.com/github/TomasContrerasMA/CienciaDatos_Tom_Diplomado/blob/main/TP1/Data_Analyst_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Analyst Assignment

## Introduction

You are working with a US retail customer on a pilot deployment.  They are using technology to track their merchandise throughout their supply chain.  The flow of their supply is:

*   **DC 1:**  Orders are filled and palletized.
*   **Truck:** Pallets travel from the DC 1 to DC 2 via semi-truck.
*   **DC 2:**  Pallets are unloaded, and additional merchandise may be added.  They are then reloaded onto a new truck.
*   **Truck:** Pallets travel from DC 2 to the Store.
*   **Store:** Pallets are unloaded, cases are removed, and stocked, and the empty cases are left behind the building awaiting pickup.

Your job is to dig into the data and find compelling insights to show the value fo the technology and help move the contract from a pilot into a full scaled deployment.



---

## Part 0: Imports

Import necessary packages and

In [1]:
import pandas as pd

In [3]:
A=pd.read_excel('/content/Assignment_1.xlsx')

In [23]:
A

,asset_type,asset_id,tag_id,Site,Zone,device_id,time_est,lat,lng,Temperature_C,Temperature_F
0,tote,2,(01)00850027865010(21)0082T0219,DC 1,dock_doors_DC1,7F9A8353E973,2022-08-02 11:59:26.628,47.79158,-65.68902,23.0,73.40
1,tote,2,(01)00850027865010(21)0082T0219,DC 1,dock_doors_DC1,EC5B0499234F,2022-08-02 12:00:22.660,47.79158,-65.68902,23.0,73.40
2,tote,2,(01)00850027865010(21)0082T0219,DC 1,dock_doors_DC1,7F9A8353E973,2022-08-02 12:01:11.234,47.79158,-65.68902,23.0,73.40
3,tote,2,(01)00850027865010(21)0082T0219,DC 1,dock_doors_DC1,3D8B2BDB8673,2022-08-02 13:47:58.172,47.79158,-65.68902,23.5,74.30
4,tote,2,(01)00850027865010(21)0082T0670,DC 1,dock_doors_DC1,7F9A8353E973,2022-08-02 11:58:55.049,47.79158,-65.68902,23.0,73.40
...,...,...,...,...,...,...,...,...,...,...,...
130056,tote,205,(01)00850027865010(21)0082T1137,Store,store_front_Store,26AB1C9A2E41,2022-08-04 12:48:51.273,47.74923,-65.64101,23.1,73.58
130057,tote,205,(01)00850027865010(21)0082T1137,Store,store_front_Store,26AB1C9A2E41,2022-08-04 12:49:58.699,47.74923,-65.64101,23.5,74.30
130058,tote,205,(01)00850027865010(21)0082T1137,Store,store_front_Store,26AB1C9A2E41,2022-08-04 12:52:45.173,47.74923,-65.64101,23.0,73.40
130059,tote,205,(01)00850027865010(21)0082T1137,Store,store_front_Store,26AB1C9A2E41,2022-08-04 12:53:48.228,47.74923,-65.64101,23.0,73.40


### Dataset Overview

* Site:  A large space that could contain multiple readers. Ex: DC 1.
* Zone:  Point of interest. These represent areas in which repeaters are installed. These can be thought of as sub-zones.  Ex: Dock Doors.
* Asset ID: The unique ID of the asset.
* Asset Type: The type of thing that is detected (ie tote).
* Device ID: The unique gateway reader ID that detected the device in the zone (there can be multiple in one zone).
* Time est: The time in EST.
* Lon: Longituge
* Lat: Latitude
* Temperature_C / F: Temperature in Celsius, Fahrentheit

## PART 1: Data Overview

In [4]:
A.columns

Index(['asset_type', 'asset_id', 'tag_id', 'Site', 'Zone', 'device_id',
       'time_est', 'lat', 'lng', 'Temperature_C', 'Temperature_F'],
      dtype='object')

### Question 1:

* How many unique cases were we tracking throughout this pilot. (1 pt)
* What are the unique zones we could see (1 pt)
* How many POI's are in each Zone. (2 pts)


In [5]:
print('1. Casos únicos rastreados:', len(A.asset_id.unique()))

1. Casos únicos rastreados: 18


In [6]:
print('Zonas únicas:')
for i in A['Site'].unique(): print(i)

Zonas únicas:
DC 1
DC 2
Transit
Store


In [7]:
a1=A['device_id'].nunique()
print('POI\'s:',a1,'\n','ZONAS ÚNICAS:')

for i in A['Zone'].unique(): print(i)

POI's: 38 
 ZONAS ÚNICAS:
dock_doors_DC1
dock_doors_DC2
Forklift3_DC1
pallet_assembly_DC1
PhoneKit1Bridge
PhoneKit2Bridge
PhoneKit2GW
point_of_sale_Store
receiving_Store
staging_DC1
staging_DC2
store_back_Store
store_front_Store
Forklift1_DC1
PhoneKit1GW


### Question 2:

* What is the temperature range we see?  (1pt)
* Where is temperature the highest and lowest (1pt)

In [8]:
mint, maxt=A.Temperature_C.agg(['min','max'])
print('Rango de temperatura en Celsius=',maxt-mint)
print('Máxima temperatura=', maxt, '\nMínima temperatura=',mint)

Rango de temperatura en Celsius= 25.0
Máxima temperatura= 44.0 
Mínima temperatura= 19.0


## Part 2: The Journey of a Case

### Question 3:

* Create a visualization that shows where a case was at over time at the zone or POI level. Imagine that this would be included in your presentation to the customer. (Non techical audience) (3 pts)

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

In [10]:
#Caja numero n
n=2

In [11]:
A1=A[A.asset_id==n]
D1=pd.DataFrame(A1.groupby(['Zone']).time_est.mean())
D2=D1.reset_index()
D2.columns

Index(['Zone', 'time_est'], dtype='object')

In [12]:
fig=px.bar(D2,x='Zone',y='time_est', text='time_est')
fig

### Question 4:

* Visualize how the temperatue changes over time along its journey.  Imagine that this would be included in your presentation to the customer. (Non techical audience) (4 pts)



In [13]:
A1=A[A.asset_id==n]
D1=pd.DataFrame(A1.groupby(['Zone']).Temperature_C.mean())
D2=D1.reset_index()

In [14]:
fig=px.bar(D2,x='Zone',y='Temperature_C', text='Temperature_C')
fig

### Question 5:
* Visualize the lon lat data on a map to show how the case traveled.  You may incorporate any other additional information to make this more impactful. Imagine that this would be included in your presentation to the customer. (Non techical audience) (5 pts)

**Do not worry if this looks like non-sense on a map.  Ex:  The trip may appear to occur over water or in a forest because this is a toy dataset.**

In [ ]:
# YOUR CODE HERE:

# Part 3: Customer Questions


### Question 6:

The customer wants to understand the efficieny of ther DC operations.
* Based on what you see in the data, (all zones except for STORE), which parts of their operation are most & least "efficient? (10 pts)

In [17]:
# YOUR CODE HERE
A1
#Tiempo de la caja #2 en el sitio "DC 1"
DC_1= A1[A1['Site'] == 'DC 1'].time_est
DC_1=DC_1.sort_values()
DC_1
DC_1_2=DC_1[20]-DC_1[22]
print('Tiempo de la caja 2 en DC_1....',DC_1_2)

Tiempo de la caja 2 en DC_1.... 0 days 02:52:30.149000


In [19]:
#Tiempo de la caja #2 en el sitio "DC 2"
DC_2= A1[A1['Site'] == 'DC 2'].time_est
DC_2=DC_2.sort_values()
DC_2
DC_2_2=DC_2[1608]-DC_2[7]
print('Tiempo que estuvo la caja 2 en DC 2 fue de....',DC_2_2)

Tiempo que estuvo la caja 2 en DC 2 fue de.... 0 days 16:39:51.467000


Por lo tanto fue más eficiente DC_1 ya que tardó menos tiempo.

### Question 7:

The customer wants to understand the stocking efficiency in stores.
* Based on what you see in the data, how quickly did the store unload and stock the merchandise. (5 pts)
* How could this be converted in a KPI that a regional manager could track?  (5 pts)

In [22]:
#Tiempo de la caja #2 en "Store"
tienda= A1[A1['Site'] == 'Store'].time_est
tienda=tienda.sort_values()
efic=tienda[880]-tienda[881]
print('Tiempo de descarga de la caja #2 en tienda...',efic)

Tiempo de descarga de la caja #2 en tienda... 1 days 04:53:08.345000


YOUR TEXT ANSWER HERE

### Question 8:

Please explain what you would ask for and what you will do with this data, given that you can talk with the following people (no code needed):


YOUR TEXT ANSWER HERE
* a. What areas concentrate the most cash flow?
* b. At what time does the highest temperature occur?

## Part 4: Bonus Insights

### Question 8

The customer is open to hearing about additional insights you found in the data above and beyond what they asked for.
* Based on what you can see in the data, are there any other interesting insights that the customer may want to hear about? (Up to 15 bonus points)



In [ ]:
# YOUR CODE HERE
# Another metrics that could be interesting would be temperature by time

YOUR TEXT ANSWER HERE
